In [3]:
from os.path import abspath
from pyspark.sql import SparkSession

# set default location for [spark-warehouse]
warehouse_location = abspath('spark-warehouse')

In [4]:
spark = (
    SparkSession
    .builder
    .appName("Delta Lake")
    .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.1.1,io.delta:delta-core_2.12:1.0.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.1")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.sql.debug.maxToStringFields","1000")
    .config("spark.kryoserializer.buffer.max","128m")
    .config("spark.driver.memory", "16G")
    .enableHiveSupport()
    .getOrCreate()
)
spark.sparkContext.setLogLevel("WARN")

In [5]:
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.project.id", "hering-datalake-prod")
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

---

In [6]:
from pyspark.sql.avro.functions import from_avro, to_avro
import pyspark.sql.functions as f
from urllib.request import urlopen
import json
from pyspark.sql.types import StringType
from dateutil import parser
from pyspark.sql import DataFrame as SparkDataFrame,DataFrame
from datetime import datetime
from typing import List
from dateutil import parser
import json
import pyspark.sql.functions as f
from pyspark.sql.types import StructField, StructType, StringType, DoubleType, BinaryType, IntegerType, TimestampType
from pyspark.sql import Column
from pyspark.sql import DataFrame as SparkDataFrame,DataFrame
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType
from delta.tables import DeltaTable
from datetime import datetime, timedelta
import os
import paramiko

In [7]:

def generete_symlink_manifest(spark:SparkSession, bucket:str) -> None:
        """
        Gera o arquivo symlink manifest para a tabela Delta do bucket de referencia
        """
        delta_table = DeltaTable.forPath(spark, bucket)
        delta_table.generate("symlink_format_manifest")


def path_exists(path, spark:SparkSession):
        """
        Dado um caminho verifica se existe ou não
        """
        sc = spark.sparkContext
        fs = sc._jvm.org.apache.hadoop.fs.FileSystem.get(
            sc._jvm.java.net.URI.create("gs://" + path.split("/")[2]),
            sc._jsc.hadoopConfiguration(),
        )
        return fs.exists(sc._jvm.org.apache.hadoop.fs.Path(path))


def last_date(delta_path:str, spark:SparkSession):
        """
        Retorna a data (timestamp_kafka) do último registro da tabela Delta
        """
        if not path_exists(delta_path + 'job_control/last_date', spark):
            try:
                df_temp = (
                    spark
                    .read
                    .format("delta")
                    .load(delta_path)
                )

                list_result = (
                    df_temp
                    .agg({"timestamp_kafka": "max"}).collect()
                )

                last_date = str(list_result[0].asDict()['max(timestamp_kafka)'])
                snapshot_flag = False
            except:
                snapshot_flag = True
                last_date = ('01-01-2021 12:00AM')

            df = spark.read.json(spark.sparkContext.parallelize([{'last_date': last_date}]))
            df.write.mode('overwrite').csv(delta_path + 'job_control/last_date', header = 'true')
        else:
            snapshot_flag = False
            df = spark.read.format("csv").load(delta_path + 'job_control/last_date', header = 'true')
            last_date = df.collect()[0].asDict()['last_date']

        return parser.parse(last_date),snapshot_flag


def update_last_date(delta_path:str, df:DataFrame, spark:SparkSession):
        """
        Atualiza a data (timestamp_kafka) do último registro da tabela Delta
        """
        list_result = (
            df
            .agg({"timestamp_kafka": "max"}).collect()
        )

        last_date = str(list_result[0].asDict()['max(timestamp_kafka)'])
        df = spark.read.json(spark.sparkContext.parallelize([{'last_date': last_date}]))
        df.write.mode('overwrite').csv(delta_path + 'job_control/last_date', header = 'true')

        return last_date

def get_operations_counts(df:DataFrame, spark:SparkSession):
        """
        Retorna o número de registros de cada operação kafka (__op)
        """
        op_count_list =(
            df
            .groupBy('__op')
            .count()
            .collect()
        )
        
        new_data_count = 0
        updating_data_count = 0
        deleting_data_count = 0

        if len(op_count_list) > 0:
            for i in op_count_list:
                dic_temp = i.asDict()
                if (dic_temp['__op'] == 'r'):
                    new_data_count = new_data_count + dic_temp['count']
                elif (dic_temp['__op'] != 'r'):
                    updating_data_count = updating_data_count + dic_temp['count']
                elif (dic_temp['__op'] == 'd'):
                    deleting_data_count = deleting_data_count + dic_temp['count']

        new_total_data_count = new_data_count + updating_data_count + deleting_data_count
        
        return new_total_data_count, new_data_count, updating_data_count, deleting_data_count


def get_data(spark:SparkSession, bucket:str, date_colunm_name:str, date_begin:datetime, source_format:str='delta', is_landing:bool=False):
        """
        Carrega Dataframe filtrando os dados do bucket de referência conforme 
        data de início enviada no parametro

        :param str date_begin: Data de início para filtro dos dados
        """
        if is_landing:
            schema = StructType() \
                    .add("key",BinaryType(),True) \
                    .add("value",BinaryType(),True) \
                    .add("topic",StringType(),True) \
                    .add("partition",IntegerType(),True) \
                    .add("offset",StringType(),True) \
                    .add("timestamp",TimestampType(),True)
            df = (
                spark
                .read
                .format(source_format)
                .schema(schema)
                .load(bucket)
                .where(
                    (f.col("year")>date_begin.year)|
                    (
                        (f.col("year")==date_begin.year)&
                        (
                        (f.col("month")>date_begin.month)|
                        (
                            (f.col("month")==date_begin.month)&
                            (
                            (f.col("day")>date_begin.day)|
                            (
                                (f.col("day")==date_begin.day)&
                                (f.col("hour")>=date_begin.hour)&
                                (f.col(date_colunm_name)>date_begin)
                            )
                            )
                        )
                        )          
                    ) 
                )
            )
        else:
            df = (
                spark
                .read
                .format(source_format)
                .load(bucket)
                .where(
                    (f.col("year")>date_begin.year)|
                    (
                        (f.col("year")==date_begin.year)&
                        (
                        (f.col("month")>date_begin.month)|
                        (
                            (f.col("month")==date_begin.month)&
                            (
                            (f.col("day")>date_begin.day)|
                            (
                                (f.col("day")==date_begin.day)&
                                (f.col("hour")>=date_begin.hour)&
                                (f.col(date_colunm_name)>date_begin)
                            )
                            )
                        )
                        )          
                    ) 
                )
            )
        return df


def write_new_data(spark:SparkSession, df:DataFrame, bucket:str, destination_format:str='delta'):
        """
        Escreve no bucket de destino os dados novos, cuja a operação kafka é 'r'
        """
        df_r = df.where((f.col('__op') == 'r'))
        (
            df_r
            .write
            .format(destination_format)
            .mode("append")
            .option("checkpointLocation", bucket + "checkpoint")
            .save(bucket)
        )
        
        return True


def update_new_data(spark:SparkSession, df:DataFrame, bucket:str, pk:list=['pk_single']):
        """
        Escreve no bucket de destino os dados novos, cuja a operação kafka é 'c' e
        atualiza os dados existentes, cuja a operação kafka é 'u' ou 'd'
        """
        table = DeltaTable.forPath(spark, bucket)
        df_table_u = df.where((f.col('__op') != 'r'))
        df_table_u_unique = df_table_u.orderBy('timestamp_kafka', ascending = False).dropDuplicates(pk)
        merge_string = 'destination.' + pk[0] + ' = updates.' + pk[0]
        (
            table.alias('destination')
            .merge(df_table_u_unique.alias('updates'), merge_string)
            .whenMatchedUpdateAll()
            .whenNotMatchedInsertAll()
            .execute()
        )
        
        return True


def delete_new_data(spark:SparkSession, bucket:str):
        """
        Deleta da tabela delta, no bucket de destino, os dados cuja a operação kafka é 'd'
        """
        table = DeltaTable.forPath(spark, bucket)
        table.delete(f.col('__op') == 'd')
        
        return True


def flatten_df(nested_df):
        """
        Retorna um dataframe com as colunas de um dataframe aninhado
        """
        flat_cols = [c[0] for c in nested_df.dtypes if c[1][:6] != 'struct']
        nested_cols = [c[0] for c in nested_df.dtypes if c[1][:6] == 'struct']

        flat_df = nested_df.select(flat_cols +
                                [f.col(nc+'.'+c)
                                    for nc in nested_cols
                                    for c in nested_df.select(nc+'.*').columns])
        return flat_df


def write_bronze_table(df:DataFrame, bucket:str):
        """
        Escreve no bucket de destino a tabela delta bronze
        """
        (
            df
            .write
            .partitionBy("year", "month", "day", "hour")
            .format("delta")
            .mode("append")
            .option("checkpointLocation", bucket + "checkpoint")
            .save(bucket)
        )

        return True

def get_schema(df:DataFrame):
    json_str = df.limit(1).toPandas()['fixedValue'][0]
    json_fields = json.loads(json_str)['schema']['fields']
    struct_fields = []
    for field in json_fields:
        struct_fields.append(StructField(field['field'], StringType()))
        
    return StructType(struct_fields)

def decode_data_frame(df:DataFrame, schema:StructType):
    df_decoded = ( 
        df
        .withColumn('fixedValue', f.col("value").cast("string"))
        .select(f.get_json_object('fixedValue',"$.payload").alias('payload'), 'timestamp_kafka')
        .withColumnRenamed('timestamp', 'timestamp_kafka')
        .select(f.from_json(f.col('payload'), schema).alias("DF"), 'timestamp_kafka')
        .select("DF.*" , 'timestamp_kafka')
    )
    
    return df_decoded

In [8]:

#PATH SILVER ZONE
SILVER_PATH_VENDA = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/venda/'
SILVER_PATH_VENDA_ITEM = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/venda_item/'
SILVER_PATH_FILIAL = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/filial/'
SILVER_PATH_REDE_LOJA = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/rede_loja/'
SILVER_PATH_EMPRESA = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/empresa/'
SILVER_PATH_PRODUTO_BARRA = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/mb_produtos_barra/'
SILVER_PATH_VENDEDOR_LOJA = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/vendedor_loja/'
SILVER_PATH_FILIAL_PONTO = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/filial_ponto/'
SILVER_PATH_SITE = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/site/'
SILVER_PATH_GERENTE_REGIONAL = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/gerente_regional/'
SILVER_PATH_CONSULTOR = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/consultor/'

SILVER_PATH_PRODUTOS =  'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/linx_db/dbo/produtos/'
SILVER_PATH_PRODUTOS_BARRA_LINX = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/linx_db/dbo/produtos_barra/'
SILVER_PATH_HR_WEB_CATALOGO_SKU =  'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/linx_db/dbo/hr_web_catalogo_sku/'
SILVER_PATH_HR_WEB_CATALOGO_FOTO =  'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/linx_db/dbo/hr_web_catalogo_foto/'
SILVER_PATH_PRODUTO_CORES =  'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/linx_db/dbo/produto_cores/'

SILVER_PATH_BASE_CLIENTES =  'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher361/db_analise/crm/tb_base_clientes/'



#PATH GOLD ZONE
GOLD_PATH_CUSTOMERS = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/homol/customers'
GOLD_PATH_ORDERS = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/homol/orders/'
GOLD_PATH_ORDERS_ITEMS = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/homol/orders_items/'
GOLD_PATH_PRODUCTS = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/homol/products/'
GOLD_PATH_SELLERS = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/homol/sellers/'
GOLD_PATH_STORES = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/homol/stores/'


GOLD_PATH_OTO_DZARM = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/homol/dzarm/'
GOLD_PATH_OTO_HERING_HMG = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/homol/hering_hmg/'
GOLD_PATH_OTO_HERING_KIDS = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/homol/hering_kids/'
GOLD_PATH_OTO_HERING_OUTLETS = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/homol/hering_outlets/'

In [9]:
def send_to_ftp(df:DataFrame, brand:str, table_name:str):
    
    file_name = brand + '_' + table_name + '_' + datetime.today().strftime("%Y%m%d_%H%M%S") + '.temp'

    df.coalesce(1).write.option("header",True).option("delimiter","|").csv("./" + file_name)
    
    for file in os.listdir(file_name):
        try:
            if file.split('.')[1] == 'csv':
                old_file = file
        except:
            pass
    
    os.rename(file_name + '/' + old_file, file_name.split('.')[0] + '.csv')
    
    file_name = file_name.split('.')[0] + '.csv'

    if brand == 'heringhomolog':
        username = 'sftp_1izmuiw'
        password = 'MXC3DY2fGyTyCYry'
    elif brand == 'heringkids':
        username = 'sftp_npedplr'
        password = 'CK0gCOGc07oRVsEh'
    elif brand == 'heringoutlet':
        username = 'sftp_h8rl1pl'
        password = 'VJojx3686H9z5T2M'
    elif brand == 'dzarm':
        username = 'sftp_yue6bjj'
        password = 'i4ICSxGv8pSqNyw5'
   
    host = 'amz2.pmweb.com.br'
    port = 22722
        
    paramiko.util.log_to_file("paramiko.log")
    transport = paramiko.Transport((host,port))
    transport.connect(None,username,password)
    sftp = paramiko.SFTPClient.from_transport(transport)
        
    # Upload
    filepath = "/data/" + file_name
    localpath = file_name
    sftp.put(localpath,filepath)
    
    # Close
    if sftp: sftp.close()
    if transport: transport.close()
        
    return None

In [10]:
filial = DeltaTable.forPath(spark, SILVER_PATH_FILIAL)
rede_loja = DeltaTable.forPath(spark, SILVER_PATH_REDE_LOJA)
filial_ponto = DeltaTable.forPath(spark, SILVER_PATH_FILIAL_PONTO)
empresa = DeltaTable.forPath(spark, SILVER_PATH_EMPRESA)
site = DeltaTable.forPath(spark, SILVER_PATH_SITE)
gerente_regional = DeltaTable.forPath(spark, SILVER_PATH_GERENTE_REGIONAL)
consultor = DeltaTable.forPath(spark, SILVER_PATH_CONSULTOR)

In [11]:
filial.toDF().createOrReplaceTempView("view_filial")
rede_loja.toDF().createOrReplaceTempView("view_rede_loja")
filial_ponto.toDF().createOrReplaceTempView("view_filial_ponto")
empresa.toDF().createOrReplaceTempView("view_empresa")
site.toDF().createOrReplaceTempView("view_site")
gerente_regional.toDF().createOrReplaceTempView("view_gerente_regional")
consultor.toDF().createOrReplaceTempView("view_consultor")

In [35]:
sql = '''
-----HERING KIDS
------------------------------------------------------------------
SELECT               
    FILIAL.CODIGO_FILIAL AS STORE_ID,
    REPLACE(REPLACE(CASE
                    WHEN  RTRIM(LTRIM(FILIAL.CODIGO_FILIAL)) = '84' THEN CONCAT(IFNULL(RTRIM(LTRIM(FILIAL.NOME_FILIAL)),''), ' ', IFNULL(RTRIM(LTRIM(SITE.DESC_SITE)),''))
                    WHEN REDE_LOJA.DESC_REDE_LOJA IS NOT NULL AND FILIAL.NOME_SHOPPING  IS NOT NULL THEN CONCAT(RTRIM(LTRIM(REDE_LOJA.DESC_REDE_LOJA)), ' ', RTRIM(LTRIM(FILIAL.NOME_SHOPPING)))
                    ELSE FILIAL.NOME_FILIAL
            END, "''", ''), '"', '') AS STORE_NAME,
    REPLACE(REPLACE(CASE
                    WHEN  RTRIM(LTRIM(FILIAL.CODIGO_FILIAL)) = '84' THEN CONCAT(IFNULL(RTRIM(LTRIM(FILIAL.NOME_FILIAL)),''), ' ', IFNULL(RTRIM(LTRIM(SITE.DESC_SITE)),''))
                    WHEN (REDE_LOJA.DESC_REDE_LOJA IS NOT NULL AND FILIAL.NOME_SHOPPING  IS NOT NULL) THEN CONCAT(RTRIM(LTRIM(REDE_LOJA.DESC_REDE_LOJA)), ' ', RTRIM(LTRIM(FILIAL.NOME_SHOPPING)))
                    ELSE FILIAL.NOME_FILIAL
            END, "''", ''), '"', '') AS STORE_NAME_OTO,
    CASE WHEN FILIAL.CODIGO_FILIAL IN ('84', '355', '407') THEN 'E' ELSE 'L' END AS STORE_TYPE,
    '' AS STORE_EMAIL,
    DESC_REGIAO_VENDAS AS STORE_CLUSTER,
    'BRASIL' AS STORE_COUNTRY,
    FILIAL.UF AS  STORE_STATE,
    FILIAL.CIDADE AS STORE_CITY,
    '' AS STORE_NEIGHBORHOOD,
    CASE WHEN FILIAL.DATA_FECHAMENTO IS NOT NULL THEN 1 ELSE 0 END  AS   STORE_STATUS,
    FILIAL.CNPJ  AS  CUSTOM_FIELD_01,
    CONSULTOR.SIGLA_CONSULTOR  AS  CUSTOM_FIELD_02,
    IFNULL(EMPRESA.NOME,EMPRESA.DESC_EMPRESA)  AS  CUSTOM_FIELD_03,
    'HERING KIDS' AS MARCA 
    ,FILIAL.__op
    ,FILIAL.timestamp_kafka
FROM  view_filial AS FILIAL
INNER JOIN view_rede_loja AS REDE_LOJA 
    ON REDE_LOJA.ID_REDE_LOJA = FILIAL.ID_REDE_LOJA
INNER JOIN view_filial_ponto AS FILIAL_PONTO
    ON FILIAL.ID_FILIAL=FILIAL_PONTO.ID_FILIAL
INNER JOIN view_empresa AS EMPRESA  ON FILIAL.ID_EMPRESA=EMPRESA.ID_EMPRESA
LEFT JOIN view_site AS SITE
    ON FILIAL_PONTO.CODIGO_SITE_FILTRO =SITE.CODIGO_SITE 
LEFT JOIN view_gerente_regional AS GERENTE_REGIONAL ON  FILIAL.ID_GERENTE_REGIONAL= GERENTE_REGIONAL.ID_GERENTE
LEFT JOIN view_consultor AS CONSULTOR ON  FILIAL.ID_CONSULTOR= CONSULTOR.ID_CONSULTOR
WHERE (REDE_LOJA.DESC_REDE_LOJA ='HERING KIDS'      OR (FILIAL_PONTO.ID_FILIAL='10084' and FILIAL_PONTO.CODIGO_SITE_FILTRO ='004'))

UNION ALL

----HERING
------------------------------------------------------------------
SELECT               
    FILIAL.CODIGO_FILIAL AS STORE_ID,
    REPLACE(REPLACE(CASE
                    WHEN  RTRIM(LTRIM(FILIAL.CODIGO_FILIAL)) = '84' THEN CONCAT(IFNULL(RTRIM(LTRIM(FILIAL.NOME_FILIAL)),''), ' ', IFNULL(RTRIM(LTRIM(SITE.DESC_SITE)),''))
                    WHEN REDE_LOJA.DESC_REDE_LOJA IS NOT NULL AND FILIAL.NOME_SHOPPING  IS NOT NULL THEN CONCAT(RTRIM(LTRIM(REDE_LOJA.DESC_REDE_LOJA)), ' ', RTRIM(LTRIM(FILIAL.NOME_SHOPPING)))
                    ELSE FILIAL.NOME_FILIAL
            END, "''", ''), '"', '') AS STORE_NAME,
    REPLACE(REPLACE(CASE
                    WHEN  RTRIM(LTRIM(FILIAL.CODIGO_FILIAL)) = '84' THEN CONCAT(IFNULL(RTRIM(LTRIM(FILIAL.NOME_FILIAL)),''), ' ', IFNULL(RTRIM(LTRIM(SITE.DESC_SITE)),''))
                    WHEN (REDE_LOJA.DESC_REDE_LOJA IS NOT NULL AND FILIAL.NOME_SHOPPING  IS NOT NULL) THEN CONCAT(RTRIM(LTRIM(REDE_LOJA.DESC_REDE_LOJA)), ' ', RTRIM(LTRIM(FILIAL.NOME_SHOPPING)))
                    ELSE FILIAL.NOME_FILIAL
            END, "''", ''), '"', '') AS STORE_NAME_OTO,
    CASE WHEN FILIAL.CODIGO_FILIAL IN ('84', '355', '407') THEN 'E' ELSE 'L' END AS STORE_TYPE,
    '' AS STORE_EMAIL,
    DESC_REGIAO_VENDAS AS STORE_CLUSTER,
    'BRASIL' AS STORE_COUNTRY,
    FILIAL.UF AS  STORE_STATE,
    FILIAL.CIDADE AS STORE_CITY,
    '' AS STORE_NEIGHBORHOOD,
    CASE WHEN FILIAL.DATA_FECHAMENTO IS NOT NULL THEN 1 ELSE 0 END  AS   STORE_STATUS,
    FILIAL.CNPJ  AS  CUSTOM_FIELD_01,
    CONSULTOR.SIGLA_CONSULTOR  AS  CUSTOM_FIELD_02,
    IFNULL(EMPRESA.NOME,EMPRESA.DESC_EMPRESA)  AS  CUSTOM_FIELD_03,
    'HERING STORE' AS MARCA 
    ,FILIAL.__op
    ,FILIAL.timestamp_kafka
FROM  view_filial AS FILIAL
INNER JOIN view_rede_loja AS REDE_LOJA 
    ON REDE_LOJA.ID_REDE_LOJA = FILIAL.ID_REDE_LOJA
INNER JOIN view_filial_ponto AS FILIAL_PONTO
    ON FILIAL.ID_FILIAL=FILIAL_PONTO.ID_FILIAL
INNER JOIN view_empresa AS EMPRESA  ON FILIAL.ID_EMPRESA=EMPRESA.ID_EMPRESA
LEFT JOIN view_site AS SITE
    ON FILIAL_PONTO.CODIGO_SITE_FILTRO =SITE.CODIGO_SITE 
LEFT JOIN view_gerente_regional AS GERENTE_REGIONAL ON  FILIAL.ID_GERENTE_REGIONAL= GERENTE_REGIONAL.ID_GERENTE
LEFT JOIN view_consultor AS CONSULTOR ON  FILIAL.ID_CONSULTOR= CONSULTOR.ID_CONSULTOR
WHERE (REDE_LOJA.DESC_REDE_LOJA IN('HERING STORE','BASIC SHOP')     OR (FILIAL_PONTO.ID_FILIAL='10084' and FILIAL_PONTO.CODIGO_SITE_FILTRO ='001'))

UNION ALL

---DZARM
------------------------------------------------------------------
SELECT               
    FILIAL.CODIGO_FILIAL AS STORE_ID,
    REPLACE(REPLACE(CASE
                    WHEN  RTRIM(LTRIM(FILIAL.CODIGO_FILIAL)) = '84' THEN CONCAT(IFNULL(RTRIM(LTRIM(FILIAL.NOME_FILIAL)),''), ' ', IFNULL(RTRIM(LTRIM(SITE.DESC_SITE)),''))
                    WHEN REDE_LOJA.DESC_REDE_LOJA IS NOT NULL AND FILIAL.NOME_SHOPPING  IS NOT NULL THEN CONCAT(RTRIM(LTRIM(REDE_LOJA.DESC_REDE_LOJA)), ' ', RTRIM(LTRIM(FILIAL.NOME_SHOPPING)))
                    ELSE FILIAL.NOME_FILIAL
            END, "''", ''), '"', '') AS STORE_NAME,
    REPLACE(REPLACE(CASE
                    WHEN  RTRIM(LTRIM(FILIAL.CODIGO_FILIAL)) = '84' THEN CONCAT(IFNULL(RTRIM(LTRIM(FILIAL.NOME_FILIAL)),''), ' ', IFNULL(RTRIM(LTRIM(SITE.DESC_SITE)),''))
                    WHEN (REDE_LOJA.DESC_REDE_LOJA IS NOT NULL AND FILIAL.NOME_SHOPPING  IS NOT NULL) THEN CONCAT(RTRIM(LTRIM(REDE_LOJA.DESC_REDE_LOJA)), ' ', RTRIM(LTRIM(FILIAL.NOME_SHOPPING)))
                    ELSE FILIAL.NOME_FILIAL
            END, "''", ''), '"', '') AS STORE_NAME_OTO,
    CASE WHEN FILIAL.CODIGO_FILIAL IN ('84', '355', '407') THEN 'E' ELSE 'L' END AS STORE_TYPE,
    '' AS STORE_EMAIL,
    DESC_REGIAO_VENDAS AS STORE_CLUSTER,
    'BRASIL' AS STORE_COUNTRY,
    FILIAL.UF AS  STORE_STATE,
    FILIAL.CIDADE AS STORE_CITY,
    '' AS STORE_NEIGHBORHOOD,
    CASE WHEN FILIAL.DATA_FECHAMENTO IS NOT NULL THEN 1 ELSE 0 END  AS   STORE_STATUS,
    FILIAL.CNPJ  AS  CUSTOM_FIELD_01,
    CONSULTOR.SIGLA_CONSULTOR  AS  CUSTOM_FIELD_02,
    IFNULL(EMPRESA.NOME,EMPRESA.DESC_EMPRESA)  AS  CUSTOM_FIELD_03,
    'DZARM' AS MARCA 
    ,FILIAL.__op
    ,FILIAL.timestamp_kafka
FROM  view_filial AS FILIAL
INNER JOIN view_rede_loja AS REDE_LOJA 
    ON REDE_LOJA.ID_REDE_LOJA = FILIAL.ID_REDE_LOJA
INNER JOIN view_filial_ponto AS FILIAL_PONTO
    ON FILIAL.ID_FILIAL=FILIAL_PONTO.ID_FILIAL
INNER JOIN view_empresa AS EMPRESA  ON FILIAL.ID_EMPRESA=EMPRESA.ID_EMPRESA
LEFT JOIN view_site AS SITE
    ON FILIAL_PONTO.CODIGO_SITE_FILTRO =SITE.CODIGO_SITE 
LEFT JOIN view_gerente_regional AS GERENTE_REGIONAL ON  FILIAL.ID_GERENTE_REGIONAL= GERENTE_REGIONAL.ID_GERENTE
LEFT JOIN view_consultor AS CONSULTOR ON  FILIAL.ID_CONSULTOR= CONSULTOR.ID_CONSULTOR
WHERE (REDE_LOJA.DESC_REDE_LOJA ='DZARM'   OR (FILIAL_PONTO.ID_FILIAL='10084' and FILIAL_PONTO.CODIGO_SITE_FILTRO ='002'))

UNION ALL

----HERING OUTLET
------------------------------------------------------------------
SELECT               
    FILIAL.CODIGO_FILIAL AS STORE_ID,
    REPLACE(REPLACE(CASE
                    WHEN  RTRIM(LTRIM(FILIAL.CODIGO_FILIAL)) = '84' THEN CONCAT(IFNULL(RTRIM(LTRIM(FILIAL.NOME_FILIAL)),''), ' ', IFNULL(RTRIM(LTRIM(SITE.DESC_SITE)),''))
                    WHEN REDE_LOJA.DESC_REDE_LOJA IS NOT NULL AND FILIAL.NOME_SHOPPING  IS NOT NULL THEN CONCAT(RTRIM(LTRIM(REDE_LOJA.DESC_REDE_LOJA)), ' ', RTRIM(LTRIM(FILIAL.NOME_SHOPPING)))
                    ELSE FILIAL.NOME_FILIAL
            END, "''", ''), '"', '') AS STORE_NAME,
    REPLACE(REPLACE(CASE
                    WHEN  RTRIM(LTRIM(FILIAL.CODIGO_FILIAL)) = '84' THEN CONCAT(IFNULL(RTRIM(LTRIM(FILIAL.NOME_FILIAL)),''), ' ', IFNULL(RTRIM(LTRIM(SITE.DESC_SITE)),''))
                    WHEN (REDE_LOJA.DESC_REDE_LOJA IS NOT NULL AND FILIAL.NOME_SHOPPING  IS NOT NULL) THEN CONCAT(RTRIM(LTRIM(REDE_LOJA.DESC_REDE_LOJA)), ' ', RTRIM(LTRIM(FILIAL.NOME_SHOPPING)))
                    ELSE FILIAL.NOME_FILIAL
            END, "''", ''), '"', '') AS STORE_NAME_OTO,
    CASE WHEN FILIAL.CODIGO_FILIAL IN ('84', '355', '407') THEN 'E' ELSE 'L' END AS STORE_TYPE,
    '' AS STORE_EMAIL,
    DESC_REGIAO_VENDAS AS STORE_CLUSTER,
    'BRASIL' AS STORE_COUNTRY,
    FILIAL.UF AS  STORE_STATE,
    FILIAL.CIDADE AS STORE_CITY,
    '' AS STORE_NEIGHBORHOOD,
    CASE WHEN FILIAL.DATA_FECHAMENTO IS NOT NULL THEN 1 ELSE 0 END  AS   STORE_STATUS,
    FILIAL.CNPJ  AS  CUSTOM_FIELD_01,
    CONSULTOR.SIGLA_CONSULTOR  AS  CUSTOM_FIELD_02,
    IFNULL(EMPRESA.NOME,EMPRESA.DESC_EMPRESA)  AS  CUSTOM_FIELD_03,
    'HERING OUTLET' AS MARCA 
    ,FILIAL.__op
    ,FILIAL.timestamp_kafka
FROM  view_filial AS FILIAL
INNER JOIN view_rede_loja AS REDE_LOJA 
    ON REDE_LOJA.ID_REDE_LOJA = FILIAL.ID_REDE_LOJA
INNER JOIN view_filial_ponto AS FILIAL_PONTO
    ON FILIAL.ID_FILIAL=FILIAL_PONTO.ID_FILIAL
INNER JOIN view_empresa AS EMPRESA  ON FILIAL.ID_EMPRESA=EMPRESA.ID_EMPRESA
LEFT JOIN view_site AS SITE
    ON FILIAL_PONTO.CODIGO_SITE_FILTRO =SITE.CODIGO_SITE 
LEFT JOIN view_gerente_regional AS GERENTE_REGIONAL ON  FILIAL.ID_GERENTE_REGIONAL= GERENTE_REGIONAL.ID_GERENTE
LEFT JOIN view_consultor AS CONSULTOR ON  FILIAL.ID_CONSULTOR= CONSULTOR.ID_CONSULTOR
WHERE (REDE_LOJA.DESC_REDE_LOJA ='HERING OUTLET'   OR (FILIAL_PONTO.ID_FILIAL='10084' and FILIAL_PONTO.CODIGO_SITE_FILTRO ='006'))
'''

In [36]:
result_sql = spark.sql(sql)

In [37]:
bucket = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/view/stores/'
result_sql.write.format('delta').mode("append").option("checkpointLocation", bucket + "checkpoint").save(bucket)

In [14]:
her_store = result_sql.where(result_sql.MARCA == 'HERING STORE')
her_kids = result_sql.where(result_sql.MARCA == 'HERING KIDS')
her_outlet = result_sql.where(result_sql.MARCA == 'HERING OUTLET')
dzarm = result_sql.where(result_sql.MARCA == 'DZARM')

In [13]:
try:
    delta_her_store = DeltaTable.forPath(spark, GOLD_PATH_OTO_HERING_HMG + 'stores/')
    delta_her_kids = DeltaTable.forPath(spark, GOLD_PATH_OTO_HERING_KIDS + 'stores/')
    delta_her_outlet = DeltaTable.forPath(spark, GOLD_PATH_OTO_HERING_OUTLETS + 'stores/')
    delta_dzarm = DeltaTable.forPath(spark, GOLD_PATH_OTO_DZARM + 'stores/')
    is_incremental = True
except:
    is_incremental = False

In [14]:
is_incremental

True

In [15]:
if is_incremental:
    d = datetime.today() - timedelta(days=3)

    her_store = her_store.where(result_sql.timestamp_kafka > d)
    her_kids = her_kids.where(result_sql.timestamp_kafka > d)
    her_outlet = her_outlet.where(result_sql.timestamp_kafka > d)
    dzarm = dzarm.where(result_sql.timestamp_kafka > d)
    
    update_new_data(spark, her_store, GOLD_PATH_OTO_HERING_HMG + 'stores/', ['STORE_ID'])
    update_new_data(spark, her_kids, GOLD_PATH_OTO_HERING_KIDS + 'stores/', ['STORE_ID'])
    update_new_data(spark, her_outlet, GOLD_PATH_OTO_HERING_OUTLETS + 'stores/', ['STORE_ID'])
    update_new_data(spark, dzarm, GOLD_PATH_OTO_DZARM + 'stores/', ['STORE_ID'])
    
    # Remove as colunas desnecessárias
    
    # Envia somente os dados incrementais dos últimos 3 dias
    send_to_ftp(her_store, 'heringhomolog', 'stores')
    send_to_ftp(her_kids, 'heringkids', 'stores')
    send_to_ftp(her_outlet, 'heringoutlet', 'stores')
    send_to_ftp(dzarm, 'dzarm', 'stores')
    
else:
    write_new_data(spark, her_store, GOLD_PATH_OTO_HERING_HMG + 'stores/', 'delta')
    write_new_data(spark, her_kids, GOLD_PATH_OTO_HERING_KIDS + 'stores/', 'delta')
    write_new_data(spark, her_outlet, GOLD_PATH_OTO_HERING_OUTLETS + 'stores/', 'delta')
    write_new_data(spark, dzarm, GOLD_PATH_OTO_DZARM + 'stores/', 'delta')
    
    # Remove as colunas desnecessárias
    
    # Envia somente os dados incrementais dos últimos 3 dias
    d = datetime.today() - timedelta(days=3)

    her_store = her_store.where(result_sql.timestamp_kafka > d)
    her_kids = her_kids.where(result_sql.timestamp_kafka > d)
    her_outlet = her_outlet.where(result_sql.timestamp_kafka > d)
    dzarm = dzarm.where(result_sql.timestamp_kafka > d)
    
    send_to_ftp(her_store, 'heringhomolog', 'stores')
    send_to_ftp(her_kids, 'heringkids', 'stores')
    send_to_ftp(her_outlet, 'heringoutlet', 'stores')
    send_to_ftp(dzarm, 'dzarm', 'stores')

KeyboardInterrupt: 

In [126]:
send_to_ftp(dzarm, 'dzarm', 'stores')

NotFound: 404 GET https://storage.googleapis.com/download/storage/v1/b/hering-datalake-prod-southamerica-east1-bucket-gold-zone/o/oto%2Ftemp%2Fstores%2Fdzarm_stores_20220517_203046.temp%2Fpart-00000-97db84f6-9f26-40ca-aa35-f4e9fa2b60b3-c000.csv?generation=1652819482109281&alt=media: No such object: hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/temp/stores/dzarm_stores_20220517_203046.temp/part-00000-97db84f6-9f26-40ca-aa35-f4e9fa2b60b3-c000.csv: ('Request failed with status code', 404, 'Expected one of', <HTTPStatus.OK: 200>, <HTTPStatus.PARTIAL_CONTENT: 206>)

In [21]:
dzarm.toPandas()

STORE_ID                        STORE_NAME  \
0       84                    LOJA INTERNET    
1     1051  DZARM RUA JOAO PAULINO DAMASCENO   
2   1470              DZARM SHOPPING MORUMBI   
3     1020           DZARM SHOPPING ELDORADO   
4      991       DZARM SHOPPING HIGIENOPOLIS   
5   1404         DZARM SHOPPING CENTER NORTE   

                     STORE_NAME_OTO STORE_TYPE STORE_EMAIL    STORE_CLUSTER  \
0                    LOJA INTERNET           E                   E-COMMERCE   
1  DZARM RUA JOAO PAULINO DAMASCENO          L              NACIONAL INF DZ   
2            DZARM SHOPPING MORUMBI          L                        GERAL   
3           DZARM SHOPPING ELDORADO          L                        GERAL   
4       DZARM SHOPPING HIGIENOPOLIS          L              NACIONAL INF DZ   
5       DZARM SHOPPING CENTER NORTE          L                        GERAL   

  STORE_COUNTRY STORE_STATE                           STORE_CITY  \
0        BRASIL          SC                                 None   
1        BRASIL          MG  ALFENAS                               
2        BRASIL          SP  SAO PAULO                             
3        BRASIL          SP  SAO PAULO                             
4        BRASIL          SP  SAO PAULO                             
5        BRASIL          SP  SAO PAULO                             

  STORE_NEIGHBORHOOD  STORE_STATUS CUSTOM_FIELD_01       CUSTOM_FIELD_02  \
0                                0  00000000000000            E-COMMERCE   
1                                1  25041455000186               Aloysio   
2                                0  78876950016418  LP - SP2               
3                                1  78876950013150  LP - SP2               
4                                1  78876950012854  LP - SP1               
5                                0  78876950017228  LP - SP1               

           CUSTOM_FIELD_03  MARCA __op         timestamp_kafka  
0              CIA. HERING  DZARM    r 2022-05-12 23:39:33.328  
1  JENIVALDO JOSE DE AVILA  DZARM    r 2022-05-12 23:39:33.377  
2              CIA. HERING  DZARM    r 2022-05-12 23:39:33.396  
3              CIA. HERING  DZARM    u 2022-05-15 02:00:50.594  
4              CIA. HERING  DZARM    r 2022-05-12 23:39:33.374  
5              CIA. HERING  DZARM    r 2022-05-12 23:39:33.393

In [25]:
df = dzarm
brand = 'dzarm'
table_name = 'stores'

from google.cloud import storage

In [34]:


temp_path = '/hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/temp/' + brand + '/'

file_name = brand + '_' + table_name + '_' + datetime.today().strftime("%Y%m%d_%H%M%S")

df.coalesce(1).write.option("header",True).option("delimiter","|").csv('gs:/' + temp_path + file_name)

storage_client = storage.Client()
bucket = storage_client.get_bucket('hering-datalake-prod-southamerica-east1-bucket-gold-zone')

blobs_list = list(bucket.list_blobs(prefix='oto/temp/' + brand + '/' + file_name + '/'))

for file in blobs_list:
    try:
        if file.name.split('.')[-1] == 'csv':
            blob = file
    except:
        pass

bucket.rename_blob(blob, 'oto/temp/' + brand + '/' + file_name + '.csv')

<Blob: hering-datalake-prod-southamerica-east1-bucket-gold-zone, oto/temp/dzarm/dzarm_stores_20220518_004920.csv, 1652834997283763>

In [124]:
def send_to_ftp(df:DataFrame, brand:str, table_name:str):
    
    temp_path = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/temp/stores/'
    file_name = brand + '_' + table_name + '_' + datetime.today().strftime("%Y%m%d_%H%M%S") + '.temp'
    
    df.coalesce(1).write.option("header",True).option("delimiter","|").csv(temp_path + file_name)
    
    storage_client = storage.Client()
    bucket = storage_client.get_bucket('hering-datalake-prod-southamerica-east1-bucket-gold-zone')
    
    blobs_list = list(bucket.list_blobs(prefix='oto/temp/stores/' + file_name + '/'))
    
    for file in temp:
        try:
            if file.name.split('.')[-1] == 'csv':
                blob = file
                print (blob.name)
        except:
            pass
    
    bucket.rename_blob(blob, "mynewfile")
    os.rename(file_name + '/' + old_file, file_name.split('.')[0] + '.csv')
    
    file_name = file_name.split('.')[0] + '.csv'

    if brand == 'heringhomolog':
        username = 'sftp_1izmuiw'
        password = 'MXC3DY2fGyTyCYry'
    elif brand == 'heringkids':
        username = 'sftp_npedplr'
        password = 'CK0gCOGc07oRVsEh'
    elif brand == 'heringoutlet':
        username = 'sftp_h8rl1pl'
        password = 'VJojx3686H9z5T2M'
    elif brand == 'dzarm':
        username = 'sftp_yue6bjj'
        password = 'i4ICSxGv8pSqNyw5'
   
    host = 'amz2.pmweb.com.br'
    port = 22722
        
#     paramiko.util.log_to_file("paramiko.log")
#     transport = paramiko.Transport((host,port))
#     transport.connect(None,username,password)
#     sftp = paramiko.SFTPClient.from_transport(transport)
        
#     # Upload
#     filepath = "/data/" + file_name
#     localpath = file_name
#     sftp.put(localpath,filepath)
    
#     # Close
#     if sftp: sftp.close()
#     if transport: transport.close()
        
    return None

---------------------
heringhomolog
heringkids
heringoutlet
dzarm

host = 'amz2.pmweb.com.br'
port = '22722'

username_hering = 'sftp_1izmuiw'
password_hering = 'MXC3DY2fGyTyCYry'

username_hering_kids = 'sftp_npedplr'
password_hering_kids = 'CK0gCOGc07oRVsEh'

username_hering_outlet = 'sftp_h8rl1pl'
password_hering_outlet = 'VJojx3686H9z5T2M'

username_dzarm = 'sftp_yue6bjj'
password_dzarm = 'i4ICSxGv8pSqNyw5'


In [ ]:
import paramiko